In [1]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2025-03-18 20:31:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [         <=>        ]  41,00M  98,8KB/s    in 4m 1s   

2025-03-18 20:35:13 (174 KB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


tsc extension is similar to csv

In [2]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
drug_sample = drug_dataset["train"].shuffle(seed=420).select(range(1000))
# Peek at the first few examples
drug_sample[:3]

{'Unnamed: 0': [79880, 85281, 200842],
 'drugName': ['Blisovi Fe 1 / 20',
  'Ethinyl estradiol / norgestimate',
  'Varenicline'],
 'condition': ['Birth Control', 'Birth Control', 'Smoking Cessation'],
 'review': ['"Stops you from getting pregnant but I have gained so much weight it&#039;s crazy, my periods are basically gone and I&#039;m about to stop taking it."',
  '"Mood swings and depression. Suicidal thoughts... Not good. Does its job with regulating periods, but the side effects are terrible. Have been on it for 2 years and was just off of it for a month. Now I&#039;m back to my normal self. Glad to know Im not going crazy, it&#039;s just the pill making me like that. Definitely going to switch."',
  '"I tried Chantix even though it scared me. I was doing good until day 6 and everything on the left side of my body went numb. I haven&#039;t smoked but that scared me enough that I will go cold turkey the rest of the way."'],
 'rating': [2.0, 2.0, 5.0],
 'date': ['September 2, 2017'

In [4]:
# inspect data for this unnamed: 0
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [5]:
# Rename this unnamed: 0
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [8]:
# Normalize conditions

def filter_nones(x):
    return x["condition"] is not None

def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

drug_dataset = drug_dataset.map(lowercase_condition)
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]


Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

['left ventricular dysfunction', 'adhd', 'birth control']